# Setup

In [576]:
%pip install OpenAI markdown mdx_truly_sane_lists weasyprint python-dotenv pango pdflatex

Note: you may need to restart the kernel to use updated packages.


In [577]:
import os

from dotenv import load_dotenv
load_dotenv()  # Load the environment variables from .env file

True

Set up the OpenAI Assistant

In [578]:
from openai import OpenAI
client = OpenAI()

#  get the prompt from the file system
with open('prompt.md', 'r') as file:
    # Read the entire content of the file into a single string variable
    prompt = file.read()

# simple retrieve of the Assistant - not necessary because we are doing an update
# my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# update the assistant with the latest prompt
my_assistant = client.beta.assistants.update(
  "asst_3WQiw8VJ91XWikievImBqwrZ",
  instructions=prompt,
  name="Interview Playbook Generator",
  tools=[{"type": "retrieval"}],
  model="gpt-4-1106-preview"
  # model="gpt-3.5-turbo-1106"
)

In [579]:
# create a thread
my_thread = client.beta.threads.create()

import time

def create_run(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as John Gruber's markdown with no delimiters and with no commentary whatsoever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"


## Initial File Upload and Prompt

In [580]:
import time

# Which example to run
choice = 'medical-assistant'
#choice = 'infra-engineer'
#choice = 'customer-success-manager'
#choice = 'real-estate-litigator'

destination_directory = "examples/" + choice + "/"
html_destination = destination_directory + 'playbook.html'
pdf_destination = destination_directory + 'playbook.pdf'
markdown_destination = destination_directory + 'playbook.md'
word_destination = destination_directory + 'playbook.docx'

company_file = "examples/" + choice + "/about-company.md"

# Upload the company description
file_company_values = client.files.create(
  file=open(company_file, "rb"),
  purpose="assistants"
)

job_description_file = "examples/" + choice + "/job-description.md"
# Open the file and read the contents
with open(job_description_file, 'r') as file:
    # Read the entire content of the file into a single string variable
    job_description = file.read()

# Upload the job description file
file_job_description = client.files.create(
  file=open(job_description_file, "rb"),
  purpose="assistants"
)

# give OpenAI time to process the file
time.sleep(10)


# Priming the Pump

In [581]:
# Refresh the context to ensure it has read the files
primer_values = create_run(client,my_thread,my_assistant,"""
            Write a detailed description of the company values from the about company file, using the company name.
            """, [file_company_values.id])
print(primer_values)

```markdown
# About Pacific Wellness Group - Academic Healthcare Consortium

## Mission:
- To be the premier community network for the Pacific Wellness Group System, delivering the best patient experience with the most effective clinical outcomes.

## Vision:
- Healing humanity through science and compassion, one patient at a time.

## Values:
- Innovation: embracing innovation, change and continuous improvement to raise quality of care and shape the future of medicine.
- Connection: cultivating community, connection and care for each other to nurture trust, respect, belonging and shared purpose.
- Teamwork: collaborating across roles, ranks, units and departments to unlock possibilities for greater impact and fulfillment.
- Mentorship: nurturing and pursuing ongoing professional growth to enjoy impactful and fulfilling careers.
- Participation: freely sharing opinions and valuing diverse backgrounds and contributions to improve decision-making, outcomes and engagement.
- Action: uniti

In [582]:
# Refresh the context to ensure it has read the files and create context. Not included in final doc.
primer_job = create_run(client,my_thread,my_assistant,"""
            Write the job title contained in the job description file.
            Then, create the list of 5 technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description. Create the list of 5 behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            For each competency, list 3 or 4 subcompetencies and their descriptions to enable more deep exploration.
            """, [file_job_description.id])
print(primer_job)


```markdown
# Technical Competencies

### Clinical Procedures
Advanced expertise in carrying out clinical procedures, including assisting physicians during exams and patient treatment interventions.
- **Patient Care**: Providing direct patient care and ensuring comfort during clinical interventions.
- **Procedure Assistance**: Assisting with physician-directed procedures, upholding the highest standards of safety and precision.
- **Clinical Equipment**: Demonstrating proficiency with clinical equipment pertinent to patient treatment.
- **Health & Safety Compliance**: Adhering to health and safety protocols and best practices to ensure a safe environment for patients and staff.

### Medical Knowledge
Comprehensive understanding of medical practices and the ability to apply this knowledge effectively in a clinical setting.
- **Routine Tasks**: Performing everyday medical tasks with precision and care.
- **Policy & Procedural Knowledge**: Thorough understanding of clinic policies, procedu

# Create Introduction

In [627]:
playbook_title = create_run(client,my_thread,my_assistant,"""
            Generate the title for the playbook as a simple string
            """)
print(playbook_title)

Interview Playbook for Medical Assistant II




In [584]:
introduction = create_run(client,my_thread,my_assistant,"""
            Generate the introduction for the guide with no headings
            """)
print(intro)

This Interview Playbook is designed to equip the hiring team with a structured approach to evaluate candidates for the role of Department Attorney at Maxwell Fairview. The position of Department Attorney is a vital role within the company, requiring a blend of extensive legal expertise and strong interpersonal skills. The competencies outlined here align with the firm's values of unparalleled service, respect, and inclusion, and ensure the selection of a candidate who embodies these principles. The objectives of the interview include assessing technical skills, behavioral qualities, and overall fit with the company culture and values.




In [585]:
interview_structure = create_run(client,my_thread,my_assistant,"""
            Generate the Interview Structure with no headings. Include mention of the competencies in the interview structure.
            """)
print(interview_structure)

The interview for the Medical Assistant II position will take place over a 60 to 90-minute session and will be conducted in several parts to ensure a comprehensive assessment of both technical and behavioral competencies.

The interview will begin with an introduction and rapport-building phase, lasting approximately 10 minutes, to put the candidate at ease. This will be followed by a technical assessment, which will comprise around 30 minutes of the interview time, where candidates will be questioned on clinical procedures, medical knowledge, technical communication, BLS competency, and computer and technical skills.

Next, we will move into the behavioral assessment, another 30 minutes, where we will explore service excellence, team collaboration, professional development, ethical practice & integrity, and organizational skills. These assessments will deeply probe the candidate’s ability to integrate with Pacific Wellness Group's values and operating principles.

The session will end

# Create Evaluation Criteria

In [586]:
eval_criteria = create_run(client,my_thread,my_assistant,"""
    Generate the Evaluation Critera section.
    """)
print(eval_criteria)

To effectively evaluate the candidate for the Medical Assistant II role, we have developed a job scorecard that outlines the key competencies and provides a structured format for assessment. The scorecard ensures that each interviewer can record their evaluation and pertinent comments consistently.

| Competency                    | Assessment    | Comments by Interviewer     |
| ----------------------------- | ------------- | --------------------------- |
| Clinical Procedures           |                |                             |
| Medical Knowledge             |                |                             |
| Technical Communication       |                |                             |
| BLS Competency                |                |                             |
| Computer and Technical Skills |                |                             |
| Service Excellence            |                |                             |
| Team Collaboration            |                |   

# Create Technical Assessment

In [587]:
technical_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Competence 1: Clinical Procedures

Clinical Procedures competency encompasses a candidate’s expertise in performing a range of essential clinical tasks, directly impacting patient care and treatment outcomes. This skill is vital for a Medical Assistant II at Pacific Wellness Group, as they must proficiently aid physicians and address patient needs in a clinical environment.

#### Lead Question

> **Describe a complex clinical procedure you have assisted with. How did you prepare for it, and what steps did you take to ensure its success?**

_Look for details on the candidate’s ability to prepare, their understanding of the procedure, and their role in ensuring patient safety and positive outcomes._

#### Diving Deeper

- **Patient Care**: Can you walk me through how you approach a patient's comfort and anxiety before a procedure?
- **Procedure Assistance**: Tell me about a time when you had to improvise during a procedure due to unexpected circumstances.
- **Clinical Equipm

In [588]:
technical_assessment_2 = create_run(client,my_thread,my_assistant,"""
    Generate the 2nd Technical Assessment. Don't include the main section header.
    """)
print(technical_assessment_2)

### Technical Competence 2: Medical Knowledge

Medical Knowledge is a crucial technical competency that requires not only a well-established foundation in medical practices but also the capacity to apply this knowledge efficiently in patient care. This aligns with Pacific Wellness Group's vision of healing humanity with science and compassion.

#### Lead Question

> **Considering the wide range of medical conditions we encounter at Pacific Wellness Group, can you share an instance where your medical knowledge significantly improved a patient's outcome?**

_Look for examples that show the candidate's deep understanding of medical conditions, proactive application of knowledge, and impact on patient care._

#### Diving Deeper

- **Routine Tasks**: What techniques do you employ to ensure that routine tasks are performed with a high degree of accuracy and care?
- **Policy & Procedural Knowledge**: How do you stay informed about the latest clinic policies and ensure adherence to them?
- **S

In [589]:
technical_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Technical Communication

Technical Communication is about effectively conveying complex medical information in a clear, professional, and empathetic manner, to both patients and healthcare professionals. This skill is essential at Pacific Wellness Group to ensure the highest quality of care and patient comprehension.

#### Lead Question

> **Can you give an example of a time when you had to explain a difficult medical concept to a patient or a patient's family? How did you ensure they understood?**

_Look for clarity in communication, the ability to break down complex information, and the candidate's empathy and patience._

#### Diving Deeper

- **EMR Proficiency**: Explain how you have used an EMR to improve patient care or clinic operations.
- **Written Communication**: Describe your process for documenting clinical notes to ensure they are both thorough and clear for other healthcare providers.
- **Verbal Communication**: Share a time when your verbal com

In [590]:
technical_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: BLS Competency

BLS Competency is pivotal for a Medical Assistant II in ensuring the ability to respond swiftly and effectively to emergencies, providing life-saving procedures as per established guidelines.

#### Lead Question

> **Share your most challenging experience performing BLS in a clinical setting. What were the circumstances, and how did you ensure a successful outcome?**

_Look for the demonstration of proficiency under pressure, adherence to BLS protocols, and effective teamwork._

#### Diving Deeper

- **CPR**: Describe the CPR protocol you follow and the frequency with which you practice or renew your CPR certification.
- **Emergency Response**: Can you recount an instance where you were the first responder to a clinical emergency? What actions did you take?
- **Rescue Breaths**: Under what circumstances would you choose rescue breaths over full CPR, and what technique do you use?
- **Automated External Defibrillator (AED)**: Discuss your expe

In [591]:
technical_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Computer and Technical Skills

Computer and Technical Skills are essential for streamlining clinical operations, enhancing patient care, and ensuring accuracy in a digital healthcare environment.

#### Lead Question

> **Describe your experience with a challenging technological issue in the clinic and how you resolved it. What was the issue, and what was your approach to solving it?**

_Look for problem-solving capabilities, technical aptitude, and the ability to minimize disruptions in patient care._

#### Diving Deeper

- **Microsoft Office Suite**: Provide an example of how you've used Excel or another Office application to improve a work process in your clinic.
- **Technology Adaptation**: Tell us about a time you had to learn a new piece of technology or software quickly for your role. How did you approach this?
- **Multi-Tasking Software**: How do you manage using multiple software applications simultaneously without affecting your efficiency?
- **Data

In [592]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [593]:
behavioral_assessment_1 = create_run(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Competence 1: Service Excellence

Service Excellence involves committing oneself to provide exceptional patient experience, demonstrating Pacific Wellness Group's focus on connection and compassion. This competency is about putting the patient first and striving for the highest quality service.

#### Lead Question

> **Discuss a time when you went above and beyond to ensure a patient received exceptional care. What motivated you, and what was the result?**

_Look for actions that exemplify a strong patient focus, personalized care, and a desire to exceed standard expectations._

#### Diving Deeper

- **C-I-CARE Execution**: How do you incorporate C-I-CARE principles into your daily interactions with patients?
- **Patient-Centric Behavior**: Can you give an example of how you've modified care for a patient based on their individual needs or feedback?
- **Feedback Responsiveness**: Describe how you have changed a process or behavior in response to feedback from patients or 

In [594]:
behavioral_assessment_2 = create_run(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Team Collaboration

Team Collaboration embodies the synergy of working across various healthcare roles and departments, aligning with Pacific Wellness Group's value of teamwork to enhance patient care and facilitate clinic operations.

#### Lead Question

> **Tell us about a time when you were part of a healthcare team that faced a significant challenge. How did you contribute to overcoming it?**

_Look for the ability to work effectively with others, contribute to team goals, and overcome obstacles._

#### Diving Deeper

- **Interdisciplinary Communication**: Describe how you have effectively communicated with another department or discipline to achieve a patient care goal.
- **Collaborative Problem Solving**: Provide an example of a problem you solved as part of a team and the role you played in that process.
- **Shared Goals**: Can you discuss a time when you had to align with team members to achieve a common goal? What was the outcome?
- **Role Versatil

In [595]:
behavioral_assessment_3 = create_run(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Professional Development

Professional Development focuses on the aspiration for continuous learning and growth, reflecting Pacific Wellness Group's value of mentorship and participation. It represents an individual's commitment to enhance their skill set and contribute to their team's collective expertise.

#### Lead Question

> **How do you engage in professional development, and can you share a specific instance where this led to improvement in your work or patient care?**

_Look for a dedication to learning, application of new knowledge, and an initiative to improve professionally._

#### Diving Deeper

- **Skill Enhancement**: What methods or resources do you utilize for enhancing your clinical skills?
- **Knowledge Sharing**: How have you contributed to the professional growth of your colleagues?
- **Mentoring Peers**: Describe your experience with mentoring a less experienced team member. What approach did you take?
- **Feedback Welcoming**: Give an 

In [596]:
behavioral_assessment_4 = create_run(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Ethical Practice & Integrity

Ethical Practice & Integrity is about upholding the highest ethical standards and demonstrating integrity in every action and decision, crucial for maintaining trust and respect within Pacific Wellness Group.

#### Lead Question

> **Can you recount a situation where your integrity was tested in the workplace? How did you handle it, and what was the outcome?**

_Look for a commitment to ethical principles, honesty in difficult situations, and accountability for actions._

#### Diving Deeper

- **Confidentiality**: Describe your approach to maintaining confidentiality of patient information.
- **Honesty in Communication**: Talk about a time when you had to be honest with a patient or colleague about a difficult or uncomfortable topic.
- **Ethical Decision-Making**: Provide an example of an ethical dilemma you've faced in the clinic and the process you followed to resolve it.
- **Professional Accountability**: Discuss a mistake y

In [597]:
behavioral_assessment_5 = create_run(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Organizational Skills

Organizational Skills reflect an individual's ability to manage tasks effectively, enhancing clinic efficiency and directly supporting Pacific Wellness Group's value of uniting diverse perspectives for the greater good.

#### Lead Question

> **Tell us about a time when your organizational skills directly contributed to improving patient care or clinic workflow.**

_Look for examples of efficient task management, solution-oriented thinking, and successful outcomes._

#### Diving Deeper

- **Prioritization**: How do you decide what tasks to prioritize when everything seems urgent?
- **Time Management**: Can you provide an example of how you managed your time effectively during an especially busy period?
- **Resource Coordination**: Describe a situation where you had to coordinate multiple resources to complete a task or project.
- **Plan Execution**: Share an experience where you had to adjust your plan due to unforeseen circumstances 

In [598]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [599]:
practical_assignment = create_run(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

For the Medical Assistant II position, the practical assignment will involve a hands-on simulation exercise to evaluate the candidate’s clinical skills, technical aptitude, and ability to communicate effectively within a healthcare setting.

The assignment objectives are to assess the candidate’s:
- Proficiency in clinical procedures under simulated conditions.
- Application of medical knowledge to a practical scenario.
- Usage of electronic medical records and other technology in patient management.
- Performance of Basic Life Support procedures with a mannequin.

During the simulation, the interviewer will evaluate the candidate's approach to patient care, adherence to safety protocols, and ability to handle unexpected challenges. This will provide an insight into their practical skills and how they translate medical knowledge into action.

The interviewer should assess the candidate's ability to stay composed, communicate clearly, and prioritize tasks effici

# Create Team Interaction

In [600]:
team_interaction = create_run(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Fit

The team fit segment of the interview process assesses how well the candidate integrates with the existing clinic team, and whether their style of communication, collaboration, and problem-solving aligns with the team dynamics as well as Pacific Wellness Group's values.

During this phase, the candidate will interact with potential team members in a less formal setting, such as a group discussion or a small project collaboration. This provides an opportunity to observe the candidate’s interaction styles, their impact on the team atmosphere, and their ability to contribute to group objectives.

Discussion points or activities may revolve around past team experiences, how the candidate approaches collaborative work, conflict resolution, and their strategies for maintaining team cohesiveness. It is beneficial to include scenarios or role-play exercises to see the candidate's real-time interaction with team members.




# Create Candidate Q&A

In [601]:
candidate_questions = create_run(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

This section outlines questions that candidates might ask during the interview and provides comprehensive answers designed to offer insights into the role, the Pacific Wellness Group, and the expectations of the Medical Assistant II position.

1. **What does a typical day look like for a Medical Assistant II at Pacific Wellness Group?**
   A typical day involves a mix of patient care, assisting physicians with clinical procedures, managing electronic medical records, and participating in team huddles to discuss patient care priorities.

2. **How does Pacific Wellness Group ensure continuous professional development for its staff?**
   We offer regular in-house training, mentorship programs, tuition reimbursement for relevant courses, and support for obtaining specialized certifications.

3. **Can you tell me more about how the clinic incorporates the Pacific Wellness Group values into daily operations?**
   Our clinic operations are grounded in our core values; innova

# Finalize the Playbook HTML

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [602]:
import markdown

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_playbook_title = markdown.markdown(playbook_title, extensions=['extra','mdx_truly_sane_lists'])
html_intro = markdown.markdown(intro, extensions=['extra','mdx_truly_sane_lists'])
html_interview_structure = markdown.markdown(interview_structure, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



## Build the HTML

Borrowed colors from this [color palette](https://color.adobe.com/PALETTE-mini-06:-Transparent-color-theme-15d8c4a6-b06a-4530-bf5b-3fc8102ccd69)

In [603]:
# Open the file and read the contents
with open('styles.css', 'r') as file:
    # Read the entire content of the file into a single string variable
    styles = file.read()


html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")


# Open the HTML file and read its content
with open('template.html', 'r') as file:
    html_template = file.read()

# Replace the placeholders with the actual variables
html_content = html_template.format(
    styles=styles,
    html_playbook_title=html_playbook_title,
    publication_date=publication_date,
    html_intro=html_intro,
    html_interview_structure=html_interview_structure,
    html_job_description=html_job_description,
    html_eval_criteria=html_eval_criteria,
    html_technical_assessment_1=html_technical_assessment_1,
    html_short_notes=html_short_notes,
    html_technical_assessment_2=html_technical_assessment_2,
    html_notes=html_notes,
    html_technical_assessment_3=html_technical_assessment_3,
    html_technical_assessment_4=html_technical_assessment_4,
    html_technical_assessment_5=html_technical_assessment_5,
    html_behavioral_assessment_1=html_behavioral_assessment_1,
    html_behavioral_assessment_2=html_behavioral_assessment_2,
    html_behavioral_assessment_3=html_behavioral_assessment_3,
    html_behavioral_assessment_4=html_behavioral_assessment_4,
    html_behavioral_assessment_5=html_behavioral_assessment_5,
    html_practical_assignment=html_practical_assignment,
    html_team_interaction=html_team_interaction,
    html_candidate_questions=html_candidate_questions
)

# Write HTML string to a file
with open(html_destination, 'w') as file:
    file.write(html_content)


# Display the Markdown-formatted text in the notebook
# from IPython.display import display, HTML
# display(HTML(html_content))


# Create the Documents

### Create PDF from HTML

In [604]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML(html_destination).write_pdf(pdf_destination, stylesheets=['styles.css'], full_fonts=True, hinting=True, presentational_hints=True)



## Create a markdown version

For use in GitHub as an example

In [637]:
notes_snippet_md = """##### Notes\n
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________
\\__________________________________________________________________________________________\n
Rating (1-5): \\__________\n\n\n"""

from datetime import datetime

# Get the current date and time
now = datetime.now()

# Format the date and time in a nice readable format
formatted_now = now.strftime("%B %d, %Y %-I:%M %p")

print(formatted_now)

playbook_body_md = f"""
### Interview Structure
{interview_structure}

### Job Description
{job_description}

## Evaluation Scorecard
{eval_criteria}

{technical_assessment_1}
{notes_snippet_md}
            
{technical_assessment_2}
{notes_snippet_md}

{technical_assessment_3}
{notes_snippet_md}

{technical_assessment_4}
{notes_snippet_md}
            
{technical_assessment_5}
{notes_snippet_md}
            
{behavioral_assessment_1}
{notes_snippet_md}

{behavioral_assessment_2}
{notes_snippet_md}
            
{behavioral_assessment_3}
{notes_snippet_md}
            
{behavioral_assessment_4}
{notes_snippet_md}
            
{behavioral_assessment_5}
{notes_snippet_md}
            
{practical_assignment}
{notes_snippet_md}

{team_interaction}
{notes_snippet_md}

{candidate_questions}
"""

print(len(playbook_body_md))

# exec_summary = create_run(client,my_thread,my_assistant,f"""
#    Write a summary for this document:
#
#    {playbook_body_md}
#    """)

exec_summary = create_run(client,my_thread,my_assistant,f"""
    Write a detailed introduction for the document. Be clear and concise and don't use flowery language.
    """)

    
print(exec_summary)

January 01, 2024 3:40 PM
32611
The Interview Playbook is a comprehensive resource for evaluating candidates for the Medical Assistant II role at Pacific Wellness Group. It provides a clear structure for the interview process, specifying the technical and behavioral competencies required for the position. The guide's purpose is to enable objective and thorough candidate evaluations, ensuring they meet the high standards of patient care and organizational culture at Pacific Wellness Group.




In [638]:
playbook_md = f"""--- 
title: {playbook_title}
author: Created by [Deliverables.ai](https://github.com/androb/deliverables)
date: {formatted_now}
--- 

## Introduction
{exec_summary}
            
{playbook_body_md}
"""


# Write playbook to a text file
with open(markdown_destination, 'w') as file:
    file.write(playbook_md)


### Create Word Document from Markdown

In [639]:
# Convert from markdown to docx
!pandoc -s {markdown_destination} -o {word_destination} --reference-doc="custom-reference.docx" --toc=True --toc-depth=2 --shift-heading-level-by -1
!pandoc -s {markdown_destination} -o {html_destination} --template=html-reference --toc=True
!pandoc --pdf-engine "weasyprint" -s {html_destination} -o {pdf_destination}

# use this extension to shift headings to differ between HTML and docx
# --shift-heading-level-by -1

# Convert from markdown to PDF via HTML
# !pandoc --pdf-engine "weasyprint" -s {markdown_destination} -o {pdf_destination} --reference-doc="custom-reference.docx" --toc=True
# !pandoc --pdf-engine "weasyprint" -s {word_destination} -o {pdf_destination}


In [ ]:
pptx_destination = destination_directory + "playbook.pptx"
!pandoc -s {markdown_destination} -o {pptx_destination}